### <b> VERSION 1 </b>


##### La idea es ahora a partir de los temas vistos en clase poder hacer feature engineering de una manera mas objetiva

In [49]:
import numpy as np
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

### Asi voy a dividir entre train y test para mi validacion

In [50]:
def mi_split (df):

    X = df.drop(['precio'], axis = 1)
    Y = df['precio']
    x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
    return x_train, x_test, y_train, y_test

# x_train y_train son mi TRAIN
# x_test y_test son mi VALIDACION

### ................................................................................................................................

### <b> Modelo N°1 - XGBoost </b>

In [51]:
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn import preprocessing

#### Para sacar provecho a las labels provincia, tipodepropiedad y ciudad usaré algún tipo de Mean Target Encoding (basado en el promedio del precio para cada grupo de valores)

In [52]:
def mean_target_encoding(train, nombrefeat, nombretarget):
        
    temp = train.groupby(nombrefeat)[nombretarget].transform(np.mean) #Saco promedio
    temp = preprocessing.scale(temp) #Normalizo
    train[nombrefeat + "_mean"] = temp #Dejo en el DF
    
    return train

In [53]:
def mean_target_decoding(x_test, nombrefeat, x_train):
    
    #temp = x_train.loc[:,[nombrefeat,nombrefeat+"_mean"]]
    #test = pd.merge(test, temp, on = nombrefeat, how='inner', indicator = True)
    #temp = temp.set_index(nombrefeat)
    #temp = temp.drop_duplicates()
    #temp = temp.T.squeeze()
  #  values = test[nombrefeat].map(temp)
  #  test[nombrefeat+"_mean"] = values
    
    nombrefeatmean = nombrefeat + "_mean"
    
    temp = x_train.loc[:,[nombrefeat,nombrefeatmean]]
    temp = temp.set_index(nombrefeat)
    temp = temp.drop_duplicates()
    temp = temp.T.squeeze()
    values = x_test[nombrefeat].map(temp)
    x_test[nombrefeatmean] = values 


# ""   temp = train.groupby(nombrefeat)[nombretarget].agg(np.mean) #Saco promedio
#    temp_test = test[nombrefeat].map(temp)
#    temp_test = preprocessing.scale(temp_test)
#    test[nombrefeat+"_mean"] = temp_test

    return x_test

In [59]:
def armar_set(train):
    
    #Puedo resolver de forma general las que son iguales para train y test
    #es decir, aquellas que no filtran informacion a los de validacion
    
    train = train.drop(['descripcion', 'titulo', 'direccion', 'fecha'], axis = 1)
    
    train['servicios'] = train['piscina'] + train['gimnasio'] + train['usosmultiples']
    
    #Elimino los residuos
    train = train.drop(['piscina', 'gimnasio','usosmultiples','escuelascercanas', 
                        'centroscomercialescercanos'], axis = 1)
    
    #Hago el split                    
    x_train, x_test, y_train, y_test = mi_split(train)
        
    x_train["precio"] = y_train

    temporal = x_train
    
    #Calculo los mean target
    x_train_mean = mean_target_encoding(x_train, "tipodepropiedad", "precio")
    x_train_mean['precio'] = y_train
    x_train_mean = mean_target_encoding(x_train_mean, "provincia", "precio")
    x_train_mean = mean_target_encoding(x_train_mean, "ciudad", "precio")
    x_train_mean.drop("precio", axis=1)
    
    #Se los asigno a los test (NO LOS CALCULO CON ELLOS!!!!!!)
    x_test = mean_target_decoding(x_test, "tipodepropiedad", temporal)
    x_test = mean_target_decoding(x_test, "provincia", temporal)
    x_test = mean_target_decoding(x_test, "ciudad", temporal)
    
    x_train_mean.drop(["tipodepropiedad", "provincia", "ciudad"], axis=1, inplace = True)
    x_test.drop(["tipodepropiedad", "provincia", "ciudad"], axis=1, inplace = True)

    
    return x_train_mean, x_test, y_train, y_test

In [60]:
x_train, x_test, y_train, y_test = armar_set(train)